<a href="https://colab.research.google.com/github/DaiVo20/Question-answering-in-BoolQ/blob/main/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install transformers

In [6]:
!pip install datasets

  Using cached datasets-1.16.1-py3-none-any.whl (298 kB)
     |████████████████████████████████| 1.1 MB 5.2 MB/s 
     |████████████████████████████████| 132 kB 46.1 MB/s 
     |████████████████████████████████| 243 kB 45.0 MB/s 
     |████████████████████████████████| 271 kB 31.5 MB/s 
     |████████████████████████████████| 192 kB 44.4 MB/s 
     |████████████████████████████████| 160 kB 42.1 MB/s 


In [7]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
from datasets import load_dataset
dataset = load_dataset('boolq')

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/952 [00:00<?, ?B/s]

Using custom data configuration default


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/6.53M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset boolq downloaded and prepared to /root/.cache/huggingface/datasets/boolq/default/0.1.0/bf0dd57da941c50de94ae3ce3cef7fea48c08f337a4b7aac484e9dddc5aa24e5. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 9427
    })
    validation: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 3270
    })
})

In [10]:
import pandas as pd

train_data = pd.DataFrame(dataset['train'])
dev_data = pd.DataFrame(dataset['validation'])
data = pd.concat([train_data, dev_data], ignore_index=True)

In [11]:
data

,question,answer,passage
0,do iran and afghanistan speak the same language,True,"Persian (/ˈpɜːrʒən, -ʃən/), also known by its ..."
1,do good samaritan laws protect those who help ...,True,Good Samaritan laws offer legal protection to ...
2,is windows movie maker part of windows essentials,True,Windows Movie Maker (formerly known as Windows...
3,is confectionary sugar the same as powdered sugar,True,"Powdered sugar, also called confectioners' sug..."
4,is elder scrolls online the same as skyrim,False,As with other games in The Elder Scrolls serie...
...,...,...,...
12692,is manic depression the same as bi polar,True,"Bipolar disorder, previously known as manic de..."
12693,was whiskey galore based on a true story,True,SS Politician was an 8000-ton cargo ship owned...
12694,are there plants on the international space st...,True,Plant research continued on the International ...
12695,does the hockey puck have to cross the line to...,True,"In ice hockey, a goal is scored when the puck ..."


In [12]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, dev_data = train_test_split(train_data, test_size=0.1, random_state=42)

In [13]:
passages_train = train_data['passage'].values
questions_train = train_data['question'].values
answers_train = train_data['answer'].values.astype(int)

passages_test = test_data['passage'].values
questions_test = test_data['question'].values
answers_test = test_data['answer'].values.astype(int)

passages_dev = dev_data['passage'].values
questions_dev = dev_data['question'].values
answers_dev = dev_data['answer'].values.astype(int)

In [14]:
import random
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

random.seed(26)
np.random.seed(26)
torch.manual_seed(26)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") 

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
model.to(DEVICE)

learning_rate = 1e-5
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=1e-8)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [15]:
def encode_data(tokenizer, questions, passages, max_length):
    input_ids = []
    attention_masks = []

    for question, passage in zip(questions, passages):
        encoded_data = tokenizer.encode_plus(question, passage, max_length=max_length, pad_to_max_length=True, truncation_strategy="longest_first")
        encoded_pair = encoded_data["input_ids"]
        attention_mask = encoded_data["attention_mask"]

        input_ids.append(encoded_pair)
        attention_masks.append(attention_mask)

    return np.array(input_ids), np.array(attention_masks)

In [16]:
# Encoding data
max_seq_length = 256
input_ids_train, attention_masks_train = encode_data(tokenizer, questions_train, passages_train, max_seq_length)
input_ids_dev, attention_masks_dev = encode_data(tokenizer, questions_dev, passages_dev, max_seq_length)
input_ids_test, attention_masks_test = encode_data(tokenizer, questions_test, passages_test, max_seq_length)

train_features = (input_ids_train, attention_masks_train, answers_train)
dev_features = (input_ids_dev, attention_masks_dev, answers_dev)
test_features = (input_ids_test, attention_masks_test, answers_test)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2227: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [17]:
batch_size = 32

train_features_tensors = [torch.tensor(feature, dtype=torch.long) for feature in train_features]
dev_features_tensors = [torch.tensor(feature, dtype=torch.long) for feature in dev_features]
test_features_tensors = [torch.tensor(feature, dtype=torch.long) for feature in test_features]

train_dataset = TensorDataset(*train_features_tensors)
dev_dataset = TensorDataset(*dev_features_tensors)
test_dataset = TensorDataset(*test_features_tensors)

train_sampler = RandomSampler(train_dataset)
dev_sampler = SequentialSampler(dev_dataset)
test_sampler = SequentialSampler(test_dataset)

train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)
dev_dataloader = DataLoader(dev_dataset, sampler=dev_sampler, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size)

In [18]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [19]:
from tqdm import tqdm
import time

epochs = 5
grad_acc_steps = 1
train_loss_values = []
dev_acc_values = []

for epoch in tqdm(range(epochs), desc="Epoch"):
    # Training
    start = time.perf_counter()
    epoch_train_loss = 0
    model.train()
    model.zero_grad()

    for step, batch in enumerate(train_dataloader):

        input_ids = batch[0].to(DEVICE)
        attention_masks = batch[1].to(DEVICE)
        labels = batch[2].to(DEVICE)     

        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels)

        loss = outputs[0]
        loss = loss / grad_acc_steps
        epoch_train_loss += loss.item()

        loss.backward()
        
        if (step+1) % grad_acc_steps == 0: # Gradient accumulation is over
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # Clipping gradients
            optimizer.step()
            model.zero_grad()

    epoch_train_loss = epoch_train_loss / len(train_dataloader)          
    train_loss_values.append(epoch_train_loss)
    
    # Evaluation
    epoch_dev_accuracy = 0
    model.eval()

    for batch in dev_dataloader:
      
        input_ids = batch[0].to(DEVICE)
        attention_masks = batch[1].to(DEVICE)
        labels = batch[2]

        with torch.no_grad():
            outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)
                        
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        
        predictions = np.argmax(logits, axis=1).flatten()
        labels = labels.numpy().flatten()
        
        epoch_dev_accuracy += np.sum(predictions == labels) / len(labels)

    epoch_dev_accuracy = epoch_dev_accuracy / len(dev_dataloader)
    dev_acc_values.append(epoch_dev_accuracy)

    end = time.perf_counter()
    print(f'Epoch {epoch + 1}/{epochs}\tTrain loss: {epoch_train_loss:.4f}\tDev accuracy: {epoch_dev_accuracy:.4f}')
    print(f'Time: {end - start:.2f}s')
    
print('Finished training!')

Epoch:  20%|██        | 1/5 [12:23<49:34, 743.56s/it]

Epoch 1/5	Train loss: 0.6450	Dev accuracy: 0.6771
Time: 743.56s


Epoch:  40%|████      | 2/5 [24:47<37:11, 743.70s/it]

Epoch 2/5	Train loss: 0.5751	Dev accuracy: 0.7061
Time: 743.79s


Epoch:  60%|██████    | 3/5 [37:11<24:47, 743.68s/it]

Epoch 3/5	Train loss: 0.4776	Dev accuracy: 0.6943
Time: 743.67s


Epoch:  80%|████████  | 4/5 [49:34<12:23, 743.71s/it]

Epoch 4/5	Train loss: 0.3629	Dev accuracy: 0.7103
Time: 743.74s


Epoch: 100%|██████████| 5/5 [1:01:58<00:00, 743.74s/it]

Epoch 5/5	Train loss: 0.2697	Dev accuracy: 0.7090
Time: 743.93s
Finished training!


In [20]:
torch.save(model, '/content/drive/MyDrive/bert-base/berta-base-uncased.pt')

In [21]:
model = torch.load('/content/drive/MyDrive/bert-base/berta-base-uncased.pt')

In [22]:
# Evaluation on test set
test_accuracy = 0
num_classes = 2
model.eval()

with torch.no_grad():
    confusion_matrix = torch.zeros(num_classes, num_classes)
    for batch in test_dataloader:
      
        input_ids = batch[0].to(DEVICE)
        attention_masks = batch[1].to(DEVICE)
        labels = batch[2]
    
        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks)
                        
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        
        predictions = np.argmax(logits, axis=1).flatten()
        labels = labels.numpy().flatten()
        
        test_accuracy += np.sum(predictions == labels) / len(labels)
        
        for t, p in zip(labels, predictions):
            confusion_matrix[t, p] += 1

    test_accuracy = test_accuracy / len(test_dataloader)
    print('Accuracy:', test_accuracy)
    print('Confusion maxtrix:\n', confusion_matrix)

Accuracy: 0.7069010416666667
Confusion maxtrix:
 tensor([[ 587.,  400.],
        [ 342., 1211.]])
